In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

# Data processing - Cleaning of the dataset

In this notebook, we will clean the spotify dataset we will be using for our project. Please download the entire dataset here: https://www.kaggle.com/datasets/dhruvildave/spotify-charts **Please make sure to run 'convert_spotify_data.py' with correct paths in the method parameters before running this notebook**. 
That short script will only retain the years we acutally need for our project, in order to make the csv a lot smaller. (The dataset is too big, if we place the script in the notebook it takes a very long time to finish)

Define functions

In [2]:
# check if region has missing ranks
def checkMissingRanks(df, region, numOfRanks = 200):
    missing = 0
    missingList = []
    df_region = df[df['region'] == region]
    for index, val in df_region['date'].value_counts().items():
        if val < numOfRanks:
            missing = missing + (numOfRanks-val)
            #print('date:', index, ', #ranks:', val)
    if missing == 0:
        print('No missing ranks!')
    else:
        print(missing, 'missing rank entries!')

In [3]:
def checkMissingDates(df, region):
    df_region = df[df['region'] == region]
    
    # dates which are not in the sequence are returned
    missing = pd.date_range(start=df['date'].min(), end=df['date'].max()).difference(df_region['date'])
    
    if missing.size == 0:
        print('No missing dates!')
    else:
        print(missing.size, 'missing dates!')
        return missing

In [4]:
def checkMissingData(df, ranks = 200):
    regions = df['region'].unique()
    
    for region in regions:
        print(region)
        missingDates = checkMissingDates(df, region)
        missingRanks = checkMissingRanks(df, region, ranks)
        print()

Import dataset

In [5]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
data_raw = pd.read_csv("../data/spotify_2020+.csv")

Sort the values by region, then date and ultimately by rank for later processing

In [6]:
data_raw.sort_values(by=['region', 'date', 'rank'], inplace=True)

In [7]:
# set a new index after the sort and check if
data_raw = data_raw.reset_index(drop=True)
data_raw.head(10)

,Unnamed: 0.1,Unnamed: 0,title,rank,date,artist,url,region,chart,trend,streams
0,8620502,18739916,Whine Up,1,2020-01-01,"Nicky Jam, Anuel AA",https://open.spotify.com/track/2pWOMNQJW3g2zmG...,Andorra,viral50,SAME_POSITION,NaN
1,8620503,18739917,ROXANNE,2,2020-01-01,Arizona Zervas,https://open.spotify.com/track/696DnlkuDOXcMAn...,Andorra,viral50,MOVE_UP,NaN
2,8620504,18739918,Que Calor (with J Balvin & El Alfa),3,2020-01-01,"Major Lazer, Diplo",https://open.spotify.com/track/1G2CAJeP7rCwOZj...,Andorra,viral50,MOVE_UP,NaN
3,8620505,18739919,"Yo x Ti, Tu x Mi",4,2020-01-01,"ROSALÍA, Ozuna",https://open.spotify.com/track/7BlBVFwvbWvcwNc...,Andorra,viral50,MOVE_UP,NaN
4,8620506,18739920,Tutu,5,2020-01-01,"Camilo, Pedro Capó",https://open.spotify.com/track/1nocRtwyNPVtGcI...,Andorra,viral50,MOVE_UP,NaN
5,8620507,18739921,HIGHEST IN THE ROOM,6,2020-01-01,Travis Scott,https://open.spotify.com/track/3eekarcy7kvN4yt...,Andorra,viral50,MOVE_UP,NaN
6,8620508,18739922,Diavla,7,2020-01-01,"Chris Viz, Young Vene",https://open.spotify.com/track/1KfrcxnOLyDhd3p...,Andorra,viral50,MOVE_UP,NaN
7,8620509,18739923,Baila Conmigo (feat. Kelly Ruiz),8,2020-01-01,"Dayvi, Victor Cardenas, Kelly Ruíz",https://open.spotify.com/track/4Clmg1g8n2yBYG1...,Andorra,viral50,MOVE_UP,NaN
8,8620510,18739924,Señorita,9,2020-01-01,"Shawn Mendes, Camila Cabello",https://open.spotify.com/track/6v3KW9xbzN5yKLt...,Andorra,viral50,MOVE_UP,NaN
9,8620511,18739925,Vas A Quedarte,10,2020-01-01,Aitana,https://open.spotify.com/track/0fwIHsKXNEcb57u...,Andorra,viral50,MOVE_UP,NaN


First, lets see which columns contain missing values:

In [8]:
data_raw.isna().sum()

Unnamed: 0.1          0
Unnamed: 0            0
title                 0
rank                  0
date                  0
artist               18
url                   0
region                0
chart                 0
trend                 0
streams         2510831
dtype: int64

As evident, most missing values occur in the 'streams' column. We can drop this column since we wont need it in our analysis. We will also drop the 'Unnamed 0' column

In [9]:
data_no_streams = data_raw.drop(columns=["Unnamed: 0", "streams"],axis=1)

In [10]:
data_no_streams.head(5)

,Unnamed: 0.1,title,rank,date,artist,url,region,chart,trend
0,8620502,Whine Up,1,2020-01-01,"Nicky Jam, Anuel AA",https://open.spotify.com/track/2pWOMNQJW3g2zmG...,Andorra,viral50,SAME_POSITION
1,8620503,ROXANNE,2,2020-01-01,Arizona Zervas,https://open.spotify.com/track/696DnlkuDOXcMAn...,Andorra,viral50,MOVE_UP
2,8620504,Que Calor (with J Balvin & El Alfa),3,2020-01-01,"Major Lazer, Diplo",https://open.spotify.com/track/1G2CAJeP7rCwOZj...,Andorra,viral50,MOVE_UP
3,8620505,"Yo x Ti, Tu x Mi",4,2020-01-01,"ROSALÍA, Ozuna",https://open.spotify.com/track/7BlBVFwvbWvcwNc...,Andorra,viral50,MOVE_UP
4,8620506,Tutu,5,2020-01-01,"Camilo, Pedro Capó",https://open.spotify.com/track/1nocRtwyNPVtGcI...,Andorra,viral50,MOVE_UP


In this cell, we find out which entries have missing artist fields and inspect them:

In [11]:
artist_missing = data_no_streams.isna()
row_has_nan = artist_missing.any(axis=1)
rows_with_nan = artist_missing[row_has_nan]
indices_missing = rows_with_nan.index.values
data_no_streams.loc[indices_missing]

,Unnamed: 0.1,title,rank,date,artist,url,region,chart,trend
5715643,11122220,NO GOOD,10,2020-07-13,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,NEW_ENTRY
5715893,11142013,NO GOOD,10,2020-07-14,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5716143,11165650,NO GOOD,10,2020-07-15,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5716393,11187280,NO GOOD,10,2020-07-16,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5716643,11203201,NO GOOD,10,2020-07-17,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5716893,11230919,NO GOOD,10,2020-07-18,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5717143,11252253,NO GOOD,10,2020-07-19,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5717399,11274194,NO GOOD,13,2020-07-20,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,MOVE_DOWN
5717651,11311638,NO GOOD,14,2020-07-21,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,MOVE_DOWN
5717911,11351666,NO GOOD,19,2020-07-22,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,MOVE_DOWN


All rows seem to use the song title 'NO GOOD'. Lets check if that title is used anywhere else or if this is showing a defect data entry:

In [12]:
mask = data_no_streams["title"].str.contains("NO GOOD")
data_no_streams[mask]

,Unnamed: 0.1,title,rank,date,artist,url,region,chart,trend
5583537,3737792,NO GOOD (feat. MamboLosco),146,2021-01-06,Malerba,https://open.spotify.com/track/6ZhyRBXBHaowf3w...,Italy,top200,NEW_ENTRY
5715643,11122220,NO GOOD,10,2020-07-13,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,NEW_ENTRY
5715893,11142013,NO GOOD,10,2020-07-14,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5716143,11165650,NO GOOD,10,2020-07-15,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5716393,11187280,NO GOOD,10,2020-07-16,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5716643,11203201,NO GOOD,10,2020-07-17,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5716893,11230919,NO GOOD,10,2020-07-18,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5717143,11252253,NO GOOD,10,2020-07-19,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,SAME_POSITION
5717399,11274194,NO GOOD,13,2020-07-20,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,MOVE_DOWN
5717651,11311638,NO GOOD,14,2020-07-21,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,Japan,viral50,MOVE_DOWN


This could be an actual track name. We will therefore fill the Nan value with 'Unkown Artist' instead of deleting the rows:

In [13]:
data_no_streams["artist"].fillna(value="Unkown artist",inplace=True)

In [14]:
# sanity check
data_no_streams.isna().sum()

Unnamed: 0.1    0
title           0
rank            0
date            0
artist          0
url             0
region          0
chart           0
trend           0
dtype: int64

Check for dtypes in the dataframe

In [15]:
# check datatypes
data_no_streams.dtypes

Unnamed: 0.1     int64
title           object
rank             int64
date            object
artist          object
url             object
region          object
chart           object
trend           object
dtype: object

Convert 'date' column to date format

In [16]:
data_no_streams['date'] = pd.to_datetime(data_no_streams.date)
data_no_streams.dtypes

Unnamed: 0.1             int64
title                   object
rank                     int64
date            datetime64[ns]
artist                  object
url                     object
region                  object
chart                   object
trend                   object
dtype: object

In [17]:
# check if more cleaning is needed
# checkMissingData(data_no_streams)

Check for additional missing data is not needed because we will look into titles, where the information for missing rank is not necessary

For future work on this dataset, we are exporting the data cleaning to a new csv file.

In [18]:
data_no_streams.to_csv('../data/spotify_2020+_cleaned.csv')